In [15]:
# import libraries
import pickle
import os
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
model_path = r'/Users/Ankush/Desktop/Data Science/Seoul Bike Sharing Prediction/models/xgboost_regressor_r2_0_95_v1.pkl'

model = pickle.load(open(model_path, 'rb'))

In [3]:
# making predictions 
data = [[ 1.07807977,  0.22192612,  0.77468297,  1.82005277, -1.26200749,
       -0.25194272, -0.12916836, -0.17535463, -0.22754885, -5.47803337,
       -0.64744078, -0.43640293,  0.30459857,  1.74132735, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 ,  2.46842407,
       -0.40081071, -0.41367015],
        
       [-0.65432992,  1.24504573,  0.23697162, -0.5001855 ,  0.56154562,
       -0.28664608, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        0.94701966,  0.14260346,  0.30459857, -0.57427456,  1.71961809,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
        2.49494334, -0.41367015],
       
       [ 0.50060987,  0.98718632, -0.69180254,  2.40011233,  0.92262892,
        1.56419996, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
       -0.98911087,  0.72160984,  0.30459857, -0.57427456, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
       -0.40081071, -0.41367015]]

prediction = model.predict(data)
list(prediction)

[-74.84035, 1304.7858, 1389.761]

## Load Standard Scaler (SC)

In [4]:
sc_dump_path = r'/Users/Ankush/Desktop/Data Science/Seoul Bike Sharing Prediction/models/sc.pkl'

sc = pickle.load(open(sc_dump_path, 'rb'))

In [5]:
sc

StandardScaler()

## User Input - for prediction

In [78]:
# taking user input 
Date = '11/05/2024'
Hour = 14
Temperature = 32
Humidity = 45
Wind_Speed = 30
Visibility = 90
Solar_Radiation = 3.0
Rainfall = 0.0
Snowfall = 0.0
Seasons = 'Spring'
Holiday = 'No Holiday'
Functioning_Day = 'Yes'

## Converting User data into model consumable format

In [79]:
# creating dictionaries and creating function to convert date into day,month,year,week_day
holiday_dict = {'No Holiday': 0 ,'Holiday': 1}
functioning_day_dict = {'No': 0, 'Yes': 1}

def get_string_to_datetime(date):
    dt = datetime.strptime(date, '%d/%m/%Y')
    return {'day': dt.day, 'month': dt.month, 'year': dt.year, 'week_day': dt.strftime('%A')}

str_to_date = get_string_to_datetime(Date)
str_to_date

{'day': 11, 'month': 5, 'year': 2024, 'week_day': 'Saturday'}

In [80]:
# defining the user inputs & creating dataframe 
u_input_list = [Hour,Temperature,Humidity,Wind_Speed,Visibility,Solar_Radiation,Rainfall,Snowfall,
               holiday_dict[Holiday], functioning_day_dict[Functioning_Day], str_to_date['day'], str_to_date['month'],
               str_to_date['year']]
features_name = ['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day', 'Month', 'Year']

df_u_input = pd.DataFrame([u_input_list], columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,14,32,45,30,90,3.0,0.0,0.0,0,1,11,5,2024


In [81]:
# function to return seasons into encoding
def seasons_to_df(Seasons):
    seasons_cols = ['Spring', 'Summer', 'Winter']
    seasons_data = np.zeros((1, len(seasons_cols)), dtype = 'int')

    df_seasons = pd.DataFrame(seasons_data, columns=seasons_cols)

    if Seasons in seasons_cols:
        df_seasons[Seasons] = 1
    return df_seasons

seasons_df = seasons_to_df(Seasons)
seasons_df

,Spring,Summer,Winter
0,1,0,0


In [82]:
# function to return weekdays into enoding
def days_to_df(week_day):
    days_name = ['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_name_data = np.zeros((1,len(days_name)), dtype='int')
    
    df_days = pd.DataFrame(days_name_data, columns=days_name)
    
    if week_day in days_name:
        df_days[week_day] = 1
    return df_days

days_df = days_to_df(str_to_date['week_day'])
days_df

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0


In [83]:
df_for_pred = pd.concat([df_u_input, seasons_df, days_df], axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,14,32,45,30,90,3.0,0.0,0.0,0,1,...,2024,1,0,0,0,1,0,0,0,0


In [84]:
# scale the new data
sc_data_for_pred = sc.transform(df_for_pred)
sc_data_for_pred

array([[ 0.3562424 ,  1.59440365, -0.64291969, 27.34267366, -2.22654507,
         2.81352104, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        -0.53355074, -0.43640293, 21.8302469 ,  1.74132735, -0.58152447,
        -0.57866842, -0.40630934,  2.44399966, -0.4075004 , -0.40511677,
        -0.40081071, -0.41367015]])

# Prediction in Deployment

In [85]:
# using our model for prediction
print(f'Rented Bike Demand on date: {Date}, and time: {Hour} is : {round(model.predict(sc_data_for_pred).tolist()[0])}')

Rented Bike Demand on date: 11/05/2024, and time: 14 is : 1062
